In [1]:
from typing import *
from messages import *
import os
from features import *


In [2]:

conversations: list[Conversation] = []

for user in os.listdir(FILE_PATH):
    conversation = Conversation(os.path.join(FILE_PATH, user))
    conversations.append(conversation)


conv: Conversation

for c in conversations:
    if "moulik simp" in c.title.lower():
        conv = c
        break




In [3]:
len(conv.messages)

4473

In [ ]:
import pandas as  pd

textMessages = [msg for msg in conv.messages if msg.messageType == MessageType.TEXT]
textMessages

In [5]:
import openai
import dotenv
dotenv.load_dotenv()

True

In [6]:
df = pd.DataFrame([{"text": msg.content} for msg in textMessages])
df

,text
0,Pic
1,damn i look so happy
2,prolly cuz of reese moullik and ryan
3,Damn ok
4,bro sent the close up of me ð­ð­ð
...,...
4330,Neha sent an attachment.
4331,ayushrgarg liked a message
4332,@moulik_b i lied
4333,i wanna know


In [7]:
openai.api_key = os.getenv("OPENAI_API_KEY")

In [8]:
def get_category(text: str) -> str:

    with open("prompt.txt", "r") as f:
        prompt = f.read()

    prompt.format(df["text"][0])

    result = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "system", "content": "You are a helpful assistant who helps the user categorize their messages."}, {
            "role": "user", "content": prompt.format(text)}],
        max_tokens=100
    )

    return result.choices[0].message.content

In [9]:
# categorized: list[TextMessage, str] = []

# for i, msg in enumerate(textMessages):
#     category = get_category(msg.content)
#     categorized.append((msg, category.strip("`")))
#     print(f"{(i + 1):0>4d}/{len(textMessages)}", msg.content, category)

0001/4335 Pic `normal`
0002/4335 damn i look so happy normal


KeyboardInterrupt: 

In [10]:
import threading
from concurrent.futures import ThreadPoolExecutor, as_completed

In [11]:
def categorize_messages(textMessages: list[TextMessage]) -> list[Tuple[TextMessage, str]]:
    categorized: list[Tuple[TextMessage, str]] = []
    lock = threading.Lock()
    total = len(textMessages)

    def process_message(i: int, msg: TextMessage):
        category = get_category(msg.content).strip("`")
        with lock:
            categorized.append((msg, category))
            print(f"{(i + 1):0>4d}/{total}", msg.content, category)

    # Adjust the number of workers as needed
    max_workers = min(32, len(textMessages) or 1)

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = [
            executor.submit(process_message, i, msg)
            for i, msg in enumerate(textMessages)
        ]
        # Optionally, wait for all futures to complete
        for future in as_completed(futures):
            future.result()  # This will re-raise any exceptions encountered

    return categorized

In [12]:
categorized_messages = categorize_messages(textMessages)

    # Optionally, do something with categorized_messages
print(f"Total categorized messages: {len(categorized_messages)}")

0010/4335 ong normal
0025/4335 Chen is actually a cutie normal
0013/4335 Nah Faisal Al Faisalâs moustache hits diff normal
0027/4335 no wtd normal
0003/4335 prolly cuz of reese moullik and ryan normal
0021/4335 ayushrgarg liked a message normal
0020/4335 spittin normal
0012/4335 cuz im narcissistic and allat normal
0006/4335 this for u moullik normal
0022/4335 nehakasoju reacted ð©· to your message  normal
0002/4335 damn i look so happy normal
0028/4335 xuemiao chen normal
0007/4335 Going straight to the entire class discord normal
0005/4335 bro sent the close up of me ð­ð­ð normal
0023/4335 him and chen normal
0018/4335 He wants you bro normal
0004/4335 Damn ok normal
0024/4335 baddies **normal**
0009/4335 Yo this ta actually had the sexiest mustache freaky
0008/4335 ayushrgarg named the group Moulik simps. normal
0019/4335 Double cheeked up** freaky
0017/4335 âHeâs cheeked upâ - Moulikð freaky
0030/4335 in his lecs normal
0001/4335 Pic normal
0032/4335 Have yall se

In [ ]:
df = pd.DataFrame([{"text": msg.content, "category": category} for msg, category in categorized_messages])
df

In [17]:
df.to_csv("categorized_messages.csv", index=False)